## Vehicle type detection

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
import os
import csv
import cv2


In [35]:
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

## pip install split-folders
import splitfolders 

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img


In [3]:
input1=r"./data/Bus"
input2=r"./data/Auto"
input3=r"./data/negative"


# Pre-processing

In [6]:
i=0
for filename in os.listdir(input1):
    #path
    path=os.path.join(input1,filename)
    a=cv2.imread(path)
    
    #resize image
    resize=(280,430)
    img=cv2.resize(a,resize)
    
    #gray image
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    


In [8]:
path

'./data/Bus\\20210529_18_52_29_000_bsfUf1M1E9bMsvY4ENiWwUWjDQp2_F_4160_3120.jpg'

In [12]:
'''for dirname in os.listdir("."):
    if os.path.isdir(dirname):
        for i, filename in enumerate(os.listdir(dirname)):
            os.rename(dirname + "/" + filename, dirname + "/" + str(i) + ".bmp")'''

'for dirname in os.listdir("."):\n    if os.path.isdir(dirname):\n        for i, filename in enumerate(os.listdir(dirname)):\n            os.rename(dirname + "/" + filename, dirname + "/" + str(i) + ".bmp")'

## Rename all the images

In [34]:
'''os.getcwd()  #  Location of the current working directory
path = "C:/Users/jyoti/JRMWork/cvlab/CP_Vehicle_class/data/Auto/"
collection = path
for i, filename in enumerate(os.listdir(collection)):
    os.rename(path+ filename, path + "auto" +str(i) + ".jpg")'''

'os.getcwd()  #  Location of the current working directory\npath = "C:/Users/jyoti/JRMWork/cvlab/CP_Vehicle_class/data/Auto/"\ncollection = path\nfor i, filename in enumerate(os.listdir(collection)):\n    os.rename(path+ filename, path + "auto" +str(i) + ".jpg")'

In [35]:
'''os.getcwd()  #  Location of the current working directory

path = "C:/Users/jyoti/JRMWork/cvlab/CP_Vehicle_class/data/Bus/"
collection = path
for i, filename in enumerate(os.listdir(collection)):
    os.rename(path+ filename, path + "bus"+ str(i) + ".jpg")  '''

'os.getcwd()  #  Location of the current working directory\n\npath = "C:/Users/jyoti/JRMWork/cvlab/CP_Vehicle_class/data/Bus/"\ncollection = path\nfor i, filename in enumerate(os.listdir(collection)):\n    os.rename(path+ filename, path + "bus"+ str(i) + ".jpg")  '

#The above cells need to be executed only once
#So comment after renaming



In [7]:
input_folder = "C:/Users/jyoti/JRMWork/cvlab/CP_Vehicle_class/data/"
output = "C:/Users/jyoti/JRMWork/cvlab/CP_Vehicle_class/data/" 

# .75, .25 

splitfolders.ratio(input_folder, output, seed=10, ratio=(.8, .2))

# splitfolders.ratio(input_folder, output="cell_images2", seed=42, ratio=(.75, .25)) 
# splitfolders.ratio(input_folder, output="cell_images2", seed=42, ratio=(.7, .2, .1))
# .8  .1 .1

In [11]:
base_dir = "C:/Users/jyoti/JRMWork/cvlab/CP_Vehicle_class/data/"

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

In [12]:
train_bus_dir = os.path.join(train_dir, 'bus')  # directory with our training cat pictures
train_auto_dir = os.path.join(train_dir, 'auto')  # directory with our training dog pictures
validation_bus_dir = os.path.join(validation_dir, 'bus')  # directory with our validation cat pictures
validation_auto_dir = os.path.join(validation_dir, 'auto')  # directory with our validation dog pictures

## Check the data folders

In [13]:
num_bus_tr = len(os.listdir(train_bus_dir))
num_auto_tr = len(os.listdir(train_auto_dir))

num_bus_val = len(os.listdir(validation_bus_dir))
num_auto_val = len(os.listdir(validation_auto_dir))

total_train = num_bus_tr + num_auto_tr
total_val = num_bus_val + num_auto_val

In [20]:
print("train: ", total_train, "val: ", total_val)

train:  219 val:  73


## Data Augmentation

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.15, # .1, .2, .3
    width_shift_range=0.2,
    height_shift_range=0.2,
    #rescale=1./255,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")


In [63]:
pic = load_img('data/train/Bus/bus0.jpg')

In [64]:
pic.getpixel

<bound method Image.getpixel of <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3120x4160 at 0x1DEFC2E2310>>

#size=3120x4160

In [65]:
pic_array = img_to_array(pic)

In [66]:
pic_array.shape

(4160, 3120, 3)

In [67]:
pic_array = pic_array.reshape((1,) + pic_array.shape)


pic_array.shape

(1, 4160, 3120, 3)

In [68]:
# Generate images
count = 0
output = "data_generated/train/Bus/"
for batch in datagen.flow( pic_array, batch_size = 1, save_to_dir=output,  save_prefix='bus', save_format='jpg'):
    count += 1
    if count > 10:
        break




In [47]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

TypeError: 'tuple' object cannot be interpreted as an integer

In [ ]:
#SIFT for input1

    #initialise sift descriptor
    sift = cv2.xfeatures2d.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    
    #convert the descriptor array into a dataframe format
    out=pd.DataFrame(descriptors)
    print("descriptor shape ",i," : ", out.shape)
    i=i+1
    
    #drop first coloumn as it's the no of feature detected. Not required.
    #append to the csv file
    csv_data=out.to_csv('rik_430.csv', mode='a', header=False,index=False)

In [ ]:
i=0
for filename in os.listdir(folder2):
    
    path=os.path.join(folder2,filename)
    a=cv2.imread(path)
    
    #resize image
    resize=(280,430)
    img=cv2.resize(a,resize)
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #initialise sift descriptor
    sift = cv2.xfeatures2d.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    
    #convert the descriptor array into a dataframe format
    out=pd.DataFrame(descriptors)
    print("descriptor shape ",i," : ", out.shape)
    i=i+1
    #drop first coloumn as it's the no of feature detected. Not required.
    #append to the csv file
    csv_data=out.to_csv('bus_430.csv', mode='a', header=False,index=False)

In [ ]:
i=0
for filename in os.listdir(folder3):
    #path
    path=os.path.join(folder3,filename)
    a=cv2.imread(path)
    
    #resize image
    resize=(280,430)
    img=cv2.resize(a,resize)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #initialise sift descriptor
    sift = cv2.xfeatures2d.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    
    #convert the descriptor array into a dataframe format
    out=pd.DataFrame(descriptors)
    print("descriptor shape ",i," : ", out.shape)
    i=i+1
    
    #drop first coloumn as it's the no of feature detected. Not required.
    #append to the csv file
    csv_data=out.to_csv('negative_430.csv', mode='a', header=False,index=False)

In [ ]:
data1 = pd.read_csv(r'/CV_CP/430_all/bus_430.csv',header=None,dtype='uint8')
data1 = data1.astype(np.uint8)
data1

In [ ]:
data2= pd.read_csv(r'CV_CP/430_all/rik_430.csv',header=None,dtype='uint8')
data2=data2.astype(np.uint8)
data2

In [ ]:
data3= pd.read_csv(r'CV_CP/430_all/negative_430.csv',header=None,dtype='uint8')
data3=data3.astype(np.uint8)
data3

In [ ]:
data=data1.append(data2)
data=data.append(data3)

In [ ]:
data.columns = data.columns.astype(str)

In [ ]:
data

In [ ]:
data.to_parquet('CV_CP/430_all/df_430.parquet')

In [ ]:
csv_data=data.to_csv('CV_CP/430_all/total_430', mode='a', header=False,index=False)

# Kmeans

In [ ]:
data = pd.read_parquet("CV_CP/430_all/df_430.parquet")

In [ ]:
data

In [ ]:
kmeans = KMeans(n_clusters=5)
kmeans.fit(data)

In [ ]:
import pickle

In [ ]:
# save the model to disk
filename = 'CV_CP/430_all/Kmeans_CL_5_430_sample_Model.sav'
pickle.dump(kmeans, open(filename, 'wb'))

In [ ]:
kmeans.labels_

In [ ]:
kmeans = pickle.load(open('CV_CP/430_all/Kmeans_CL_5_430_sample_Model.sav', 'rb'))

In [ ]:
hist=np.histogram(kmeans.labels_,bins=[0,1,2,3,4,5])


print('histogram of trained kmeans')
print(hist,"\n")

In [ ]:
#performing kmeans prediction of the entire rickshaw dataset with the pretrained kmeans model

#initialising i=0; as its the first class
i=0
data=[]
#k=0

for filename in os.listdir(folder1):
    #path
    path=os.path.join(folder1,filename)
    a=cv2.imread(path)
    
    #resize image
    resize=(280,430)
    img=cv2.resize(a,resize)
    
    #gray image
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    sift = cv2.xfeatures2d.SIFT_create()
    
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    out=pd.DataFrame(descriptors)
    
    #predict values of feature vector with pretrained kmeans
    #ValueError: Buffer dtype mismatch, expected 'float', in order to avoid this dtype=np.double
    
    array_double = np.array(out, dtype=np.double)
    a=kmeans.predict(array_double)
    
    #hist= np.histogram(a,bins=[0,1,2,3,4,5,6,7,8])
    hist= np.histogram(a,bins=[0,1,2,3,4,5])
    #append the dataframe into the array in append mode, the array will only have 5 values which will store the values in a row
    data.append(hist[0])
    #k=k+1
    
#convert Array to Dataframe and append to the list
Output = pd.DataFrame(data)
#add row class 
Output["Class"] = i 
csv_data=Output.to_csv('rik_km_5_430.csv', mode='a',header=False,index=False)

In [ ]:
np.histogram(a,bins=[0,1,2,3,4,5])[0] #test

In [ ]:
#performing kmeans prediction of the entire bus dataset with the pretrained kmeans model

#initialising i=1; as its the 2nd class
i=1
data=[]
k=0
for filename in os.listdir(folder2):
      path=os.path.join(folder2,filename)
      a=cv2.imread(path)
      #resize image
      resize=(280,430)
      img=cv2.resize(a,resize)
      
      #gray image
      gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
      
      sift = cv2.xfeatures2d.SIFT_create()
      
      keypoints, descriptors = sift.detectAndCompute(gray, None)
      out=pd.DataFrame(descriptors)
      
      #predict values of feature vector with pretrained kmeans
      #ValueError: Buffer dtype mismatch, expected 'float', in order to avoid this dtype=np.double
      
      array_double = np.array(out, dtype=np.double)
      a=kmeans.predict(array_double)
      
      hist= np.histogram(a,bins=[0,1,2,3,4,5])
      
      #append the dataframe into the array in append mode, the array will only have 5 values which will store the values in a row
      data.append(hist[0])
      k=k+1
    
#convert Array to Dataframe and append to the list
Output = pd.DataFrame(data)
#add row class 
Output["Class"] = i 
csv_data=Output.to_csv('bus_km_8_430.csv', mode='a',header=False,index=False)

In [ ]:
#performing kmeans prediction of the entire negative dataset with the pretrained kmeans model

#initialising i=1; as its the 2nd class
i=2
data=[]
k=0
for filename in os.listdir(folder3):
      path=os.path.join(folder3,filename)
      a=cv2.imread(path)
      #resize image
      resize=(230,480)
      img=cv2.resize(a,resize)
      
      #gray image
      gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
      
      sift = cv2.xfeatures2d.SIFT_create()
      
      keypoints, descriptors = sift.detectAndCompute(gray, None)
      out=pd.DataFrame(descriptors)
      
      #predict values of feature vector with pretrained kmeans
      #ValueError: Buffer dtype mismatch, expected 'float', in order to avoid this dtype=np.double
      
      array_double = np.array(out, dtype=np.double)
      a=kmeans.predict(array_double)
      
      hist= np.histogram(a,bins=[0,1,2,3,4,5])
      
      #append the dataframe into the array in append mode, the array will only have 5 values which will store the values in a row
      data.append(hist[0])
      k=k+1
    
#convert Array to Dataframe and append to the list
Output = pd.DataFrame(data)
#add row class 
Output["Class"] = i 
csv_data=Output.to_csv('neg_km_5_430.csv', mode='a',header=False,index=False)

In [ ]:
csv_data=Output.to_csv('FinalNstr.csv', mode='a',header=False,index=False)

In [ ]:
#Displaying the kmeans predicted data
print("R")
dat1= pd.read_csv(r'CV_CP/430_all/rik_km_5_430.csv',header=None)
print(dat1)

In [ ]:
print("B")
dat2= pd.read_csv(r'CV_CP/430_all/bus_km_8_430.csv',header=None)
print(dat2)

In [ ]:
print("N")
dat3= pd.read_csv(r'CV_CP/430_all/neg_km_5_430.csv',header=None)
print(dat3)

In [ ]:
#appending All classes into 1 csv file

A=dat1.append(dat2)
A = A.append(dat3)
csv_data=A.to_csv('FinalFV.csv', mode='a',header=False,index=False)

In [ ]:
A = pd.read_csv("CV_CP/430_all/FinalFV.csv")

# PCA

In [ ]:
A

In [ ]:
x = A.iloc[:,0:5].values

In [ ]:
y = A.iloc[:,5].values

In [ ]:
from sklearn.preprocessing import StandardScaler
DhoniS = StandardScaler()
Dhoni = DhoniS.fit_transform(x)

In [ ]:
Dhoni

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=5)
pca.fit(Dhoni)

In [ ]:
# save the model to disk
filename = 'PCA_5_430_Model.sav'
pickle.dump(pca, open(filename, 'wb'))

In [ ]:
Bhumi = pca.transform(Dhoni)

In [ ]:
Bhumi

In [ ]:
Bhumi.shape

In [ ]:
print(pca.explained_variance_ratio_) 

In [ ]:
Bhumi = pd.DataFrame(Bhumi)

In [ ]:
Bhumi

In [ ]:
B=pd.concat([Bhumi, pd.DataFrame(y)],axis=1)

In [ ]:
csv_data=B.to_csv('FinalPCAFV5_430_5.csv', mode='a',header=False,index=False) #SAVING PCA OUTPUT

# TESTING THE DATASET

# Load Models

In [ ]:
kmeans = pickle.load(open('/content/drive/MyDrive/CV_CP/430_all/Kmeans_CL_5_430_sample_Model.sav', 'rb'))

In [ ]:
hist=np.histogram(kmeans.labels_,bins=[0,1,2,3,4,5])


print('histogram of trained kmeans')
print(hist,"\n")

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = pickle.load(open("/content/drive/MyDrive/CV_CP/430_all/PCA_5_430_SCALER.sav", 'rb'))

In [ ]:
pca = pickle.load(open("/content/drive/MyDrive/CV_CP/430_all/PCA_5_430_Model.sav", 'rb'))

In [ ]:
data= pd.read_csv(r'/content/drive/MyDrive/CV_CP/430_all/FinalPCAFV5_430_5.csv',header=None)

data.head()

In [ ]:
#assigning x the columns from 1 to 128 for training
x = data.iloc[:,0:5].values
#assigning y with the column "Class" as target variable
y = data.iloc[:,5]

In [ ]:
#Dataset split into train and test with 80% Training and 20% Testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=0)

# TEST DT

In [ ]:
def predictor(path, model):
  j = 0
  gath = []
  for i in os.listdir(path):
    if j == 200:
      break
    img=cv2.imread(os.path.join(path,i))
    resize=(280,430)

    #resize image
    #img=cv2.resize(a,resize)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    #initialise sift descriptor
    sift = cv2.xfeatures2d.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    out=pd.DataFrame(descriptors)
    #initialise Kmeans and create 5 clusters
    #test the model for the features i.e. for all elements in the Dataframe
    array_double = np.array(out, dtype=np.double)
    a=kmeans.predict(array_double)
    hist=np.histogram(a,bins=[0,1,2,3,4,5])
    #Output = pd.DataFrame([hist[0]])
    gath.append(hist[0])
    j+=1
  scaler1 = pickle.load(open("/content/drive/MyDrive/CV_CP/430_all/PCA_5_430_SCALER.sav", 'rb'))
  pca = pickle.load(open("/content/drive/MyDrive/CV_CP/430_all/PCA_5_430_Model.sav", 'rb'))
  x = scaler.fit_transform(pd.DataFrame(gath))
  y = pca.transform(x)
  y_pred1 = model.predict(y)
  #prints the prediction of the class
  return y_pred1

In [ ]:
path = folder1
model = rfc
predictor(path, model)

# DT

In [ ]:
#Decision Tree Classifier
#Assign model with Decision Tree classifier
model1 = DecisionTreeClassifier(max_depth=40)
#training the model with the Training Variables 
model1.fit(x_train, y_train)
#predicting the traget variable using testing variables
y_pred1 = model1.predict(x_test)
#Results
print("Decision Tree Results")
print("Decision Tree Accuracy: ",accuracy_score(y_test, y_pred1)*100,"%")
print("CM = \n",confusion_matrix(y_test, y_pred1))

In [ ]:
para = [] # list to save all parameter values to plot
acc = [] # list to save all accuracy values to plot
for i in range(10,200,10):
  model1 = DecisionTreeClassifier(max_depth=i)
  model1.fit(x_train, y_train) #fitting the model
  y_pred1 = model1.predict(x_test) # predicting the output
  para.append(i) #saving to list
  acc.append(accuracy_score(y_test, y_pred1)*100) #saving to list

In [ ]:
plt.plot(para,acc) #plotting accuracy vs parameter 

# SVM

In [ ]:
kernels = ['linear', 'poly', 'rbf', 'sigmoid']

In [ ]:
print("OVA")
p = dict()
for i in kernels:
  SVM = SVC(kernel=i, decision_function_shape='ovr')
  SV = SVM.fit(x_train, y_train)
  y_pred1 = SV.predict(x_test)
  #Results
  p[i] = accuracy_score(y_test, y_pred1)*100
  print("SVM " + i +" Results")
  print("SVM  " + i +" Accuracy: ",accuracy_score(y_test, y_pred1)*100,"%")
  print("CM = \n",confusion_matrix(y_test, y_pred1))
  print("\n\n")

In [ ]:
plt.bar(p.keys(),p.values())

In [ ]:
p1 = dict()
print("OVO")
for i in kernels:
  SVM = SVC(kernel=i, decision_function_shape='ovo' )
  SV = SVM.fit(x_train, y_train)
  y_pred1 = SV.predict(x_test)
  #Results
  p1[i] = accuracy_score(y_test, y_pred1)*100
  print("SVM " + i +" Results")
  print("SVM  " + i +" Accuracy: ",accuracy_score(y_test, y_pred1)*100,"%")
  print("CM = \n",confusion_matrix(y_test, y_pred1))
  print("\n\n")

In [ ]:
plt.bar(p1.keys(),p1.values())

# RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
para = [] # list to save all parameter values to plot
acc = [] # list to save all accuracy values to plot
for i in range(10,500,10):
  rfc = RandomForestClassifier(n_estimators=i)
  rfc = rfc.fit(x_train,y_train)
  y_pred1 = rfc.predict(x_test)
  para.append(i)
  acc.append(accuracy_score(y_test, y_pred1)*100)

In [ ]:
plt.plot(para,acc) #plotting accuracy vs parameter 

In [ ]:
para[acc.index(max(acc))], max(acc)

In [ ]:
rfc = RandomForestClassifier(n_estimators=350)
rfc = rfc.fit(x_train,y_train)
y_pred1 = rfc.predict(x_test)
#Results
print("RandomForestClassifier Results")
print("RandomForestClassifier Accuracy: ",accuracy_score(y_test, y_pred1)*100,"%")
print("CM = \n",confusion_matrix(y_test, y_pred1))

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
para = []
acc = []
for i in range(1,50,1):
  knc = KNeighborsClassifier(n_neighbors= i,p = 3)
  knc = knc.fit(x_train,y_train)
  y_pred1 = knc.predict(x_test)
  para.append(i)
  acc.append(accuracy_score(y_test, y_pred1)*100)

In [ ]:
plt.plot(para,acc)

In [ ]:
para[acc.index(max(acc))], max(acc)

In [ ]:
knc = KNeighborsClassifier(n_neighbors= 7,p = 3)
knc = knc.fit(x_train,y_train)

In [ ]:
y_pred1 = knc.predict(x_test)
#Results
print("KNN Results")
print("KNN Accuracy: ",accuracy_score(y_test, y_pred1)*100,"%")
print("CM = \n",confusion_matrix(y_test, y_pred1))

# GNB

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb = gnb.fit(x_train,y_train)

In [ ]:
y_pred1 = knc.predict(x_test)
#Results
print("GaussianNB Results")
print("GaussianNB Accuracy: ",accuracy_score(y_test, y_pred1)*100,"%")
print("CM = \n",confusion_matrix(y_test, y_pred1))

# TEST DT

TESTING THE BEST MODEL (Random Forest) USING FOR LOOP

In [ ]:
def predictor(path, model):
  j = 0
  gath = []
  for i in os.listdir(path):
    if j == 200:
      break
    img=cv2.imread(os.path.join(path,i))
    resize=(280,430)

    #resize image
    #img=cv2.resize(a,resize)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    #initialise sift descriptor
    sift = cv2.xfeatures2d.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    out=pd.DataFrame(descriptors)
    #initialise Kmeans and create 5 clusters
    #test the model for the features i.e. for all elements in the Dataframe
    array_double = np.array(out, dtype=np.double)
    a=kmeans.predict(array_double)
    hist=np.histogram(a,bins=[0,1,2,3,4,5])
    #Output = pd.DataFrame([hist[0]])
    gath.append(hist[0])
    j+=1
  scaler1 = pickle.load(open("/content/drive/MyDrive/CV_CP/430_all/PCA_5_430_SCALER.sav", 'rb'))
  pca = pickle.load(open("/content/drive/MyDrive/CV_CP/430_all/PCA_5_430_Model.sav", 'rb'))
  x = scaler.fit_transform(pd.DataFrame(gath))
  y = pca.transform(x)
  y_pred1 = model.predict(y)
  #prints the prediction of the class
  return y_pred1

In [ ]:
path = folder1
model = rfc
predictor(path, model)

# Dropping Negative Class

In [ ]:
data2 = data[data[5] != 2]

In [ ]:
#assigning x the columns from 1 to 128 for training
x = data2.iloc[:,0:5].values
#assigning y with the column "Class" as target variable
y = data2.iloc[:,5]

In [ ]:
#Dataset split into train and test with 80% Training and 20% Testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=0)

# DT

In [ ]:
#Decision Tree Classifier
from sklearn.metrics import accuracy_score, confusion_matrix
#Assign model with Decision Tree classifier
model1 = DecisionTreeClassifier(max_depth=40)
#training the model with the Training Variables 
model1.fit(x_train, y_train)
#predicting the traget variable using testing variables
y_pred1 = model1.predict(x_test)
#Results
print("Decision Tree Results")
print("Decision Tree Accuracy: ",accuracy_score(y_test, y_pred1)*100,"%")
print("CM = \n",confusion_matrix(y_test, y_pred1))

In [ ]:
para = [] # list to save all parameter values to plot
acc = [] # list to save all accuracy values to plot
for i in range(10,200,10):
  model1 = DecisionTreeClassifier(max_depth=i)
  model1.fit(x_train, y_train) #fitting the model
  y_pred1 = model1.predict(x_test) # predicting the output
  para.append(i) #saving to list
  acc.append(accuracy_score(y_test, y_pred1)*100) #saving to list

In [ ]:
plt.plot(para,acc)

# SVM

In [ ]:
kernels = ['linear', 'poly', 'rbf', 'sigmoid'] #making a list of kernals for easy looping

In [ ]:
from sklearn.svm import SVC #importing the required model

In [ ]:
print("OVA")
p = dict()
for i in kernels:
  SVM = SVC(kernel=i, decision_function_shape='ovr')
  SV = SVM.fit(x_train, y_train)
  y_pred1 = SV.predict(x_test)
  #Results
  p[i] = accuracy_score(y_test, y_pred1)*100
  print("SVM " + i +" Results")
  print("SVM  " + i +" Accuracy: ",accuracy_score(y_test, y_pred1)*100,"%")
  print("CM = \n",confusion_matrix(y_test, y_pred1))
  print("\n\n")

In [ ]:
plt.bar(p.keys(),p.values())

In [ ]:
p1 = dict()
print("OVO")
for i in kernels:
  SVM = SVC(kernel=i, decision_function_shape='ovo' )
  SV = SVM.fit(x_train, y_train)
  y_pred1 = SV.predict(x_test)
  #Results
  p1[i] = accuracy_score(y_test, y_pred1)*100
  print("SVM " + i +" Results")
  print("SVM  " + i +" Accuracy: ",accuracy_score(y_test, y_pred1)*100,"%")
  print("CM = \n",confusion_matrix(y_test, y_pred1))
  print("\n\n")

In [ ]:
plt.bar(p1.keys(),p1.values())

# RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
para = []
acc = []
for i in range(10,500,10):
  rfc = RandomForestClassifier(n_estimators=i)
  rfc = rfc.fit(x_train,y_train)
  y_pred1 = rfc.predict(x_test)
  para.append(i)
  acc.append(accuracy_score(y_test, y_pred1)*100)

In [ ]:
plt.plot(para,acc)

In [ ]:
para[acc.index(max(acc))], max(acc)

In [ ]:
rfc = RandomForestClassifier(n_estimators=420)
rfc = rfc.fit(x_train,y_train)
y_pred1 = rfc.predict(x_test)
#Results
print("RandomForestClassifier Results")
print("RandomForestClassifier Accuracy: ",accuracy_score(y_test, y_pred1)*100,"%")
print("CM = \n",confusion_matrix(y_test, y_pred1))

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
para = []
acc = []
for i in range(1,50,1):
  knc = KNeighborsClassifier(n_neighbors= i,p = 3)
  knc = knc.fit(x_train,y_train)
  y_pred1 = knc.predict(x_test)
  para.append(i)
  acc.append(accuracy_score(y_test, y_pred1)*100)

In [ ]:
plt.plot(para,acc)

In [ ]:
para[acc.index(max(acc))], max(acc)

In [ ]:
knc = KNeighborsClassifier(n_neighbors= 7,p = 3)
knc = knc.fit(x_train,y_train)

In [ ]:
y_pred1 = knc.predict(x_test)
#Results
print("KNN Results")
print("KNN Accuracy: ",accuracy_score(y_test, y_pred1)*100,"%")
print("CM = \n",confusion_matrix(y_test, y_pred1))

# GNB

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb = gnb.fit(x_train,y_train)

In [ ]:
y_pred1 = gnb.predict(x_test)
#Results
print("GaussianNB Results")
print("GaussianNB Accuracy: ",accuracy_score(y_test, y_pred1)*100,"%")
print("CM = \n",confusion_matrix(y_test, y_pred1))

# TEST DT

TESTING THE BEST MODEL (Random Forest) USING FOR LOOP

In [ ]:
def predictor(path, model):
  j = 0
  gath = []
  for i in os.listdir(path):
    if j == 200:
      break
    img=cv2.imread(os.path.join(path,i))
    resize=(280,430)

    #resize image
    #img=cv2.resize(a,resize)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    #initialise sift descriptor
    sift = cv2.xfeatures2d.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    out=pd.DataFrame(descriptors)
    #initialise Kmeans and create 5 clusters
    #test the model for the features i.e. for all elements in the Dataframe
    array_double = np.array(out, dtype=np.double)
    a=kmeans.predict(array_double)
    hist=np.histogram(a,bins=[0,1,2,3,4,5])
    #Output = pd.DataFrame([hist[0]])
    gath.append(hist[0])
    j+=1
  scaler1 = pickle.load(open("/content/drive/MyDrive/CV_CP/430_all/PCA_5_430_SCALER.sav", 'rb'))
  pca = pickle.load(open("/content/drive/MyDrive/CV_CP/430_all/PCA_5_430_Model.sav", 'rb'))
  x = scaler.fit_transform(pd.DataFrame(gath))
  y = pca.transform(x)
  y_pred1 = model.predict(y)
  #prints the prediction of the class
  return y_pred1

In [ ]:
path = folder1
model = rfc
predictor(path, model)